In [101]:
sim_results = [["Kavilumpara__Kozhikode", 0,1,2,3,10,20,40,80, 120,300, 600, 1020]]

In [123]:
import ipyleaflet as il


import geopandas as gp
import pandas as pd
import json
from branca.colormap import linear

shpfile = "KER_LSGD_Boundary_UTM_Modified_wgs84Radians.shp"
gdf = gp.read_file(shpfile)
#gdf = gdf[gdf['DISTRICT'] == "Kozhikode"]
gdf.loc[gdf["DISTRICT"].isnull(),"DISTRICT"] =""
gdf["UNAME"] = gdf["LSGD"] + "__" + gdf["DISTRICT"]

#It is a work around, the plotting library is still in development
idmap = gdf["UNAME"].to_dict()
propermap ={}
for key in idmap:
    propermap[str(key)] = key
    

columns =["UNAME"]
for i in range(len(sim_results[0])-1):
    columns.append("day__"+str(i))

sim_results_df = pd.DataFrame(sim_results, columns=columns)

subdf = gdf[["UNAME","geometry"]]
geojson = subdf.to_json()
geo_data = json.loads(geojson)

SLIDER_VALUE = 3

In [124]:
def get_color_for_count(x):
    if x <=0: return '#FFFFFF'
    if x <=1: return '#FCFBC0'
    if x <=4: return '#FCFB99'
    if x <=8: return '#FDFB68'
    if x <=15: return '#FEFB01'
    if x <=30: return '#FFC100'

    if x <=60: return '#FF9201'
    if x <=125: return '#FF6666'
    if x <=250: return '#FF4C4C'
    if x <=500: return '#FF3333'
    return '#FF0000'
  

def get_color(x):
    id = x['id']
    uname = idmap[int(id)]
    day = "day__"+str(SLIDER_VALUE)
    infected = 0
    if any(sim_results_df["UNAME"] == uname):
        infected = sim_results_df[sim_results_df["UNAME"] == uname][day].to_list()[0]
    lcolor = 'grey' if infected ==0 else 'black'
    return get_color_for_count(infected), lcolor


def callback(x,y,z):
    fcolor, lcolor = get_color(x)
    
    return dict(
                fillColor=fcolor,
                color='grey',
                fillOpacity=1.0,
                weight=1.0
    )

In [125]:
layer = il.Choropleth( geo_data=geo_data, choro_data=propermap, style_callback=callback) 

m = il.Map(center=(9.9816,76.2999), zoom = 8)
m.add_layer(layer)

In [126]:
def f(x):
    global SLIDER_VALUE
    SLIDER_VALUE =x
    global layer
    if layer.colormap == linear.Blues_04:
        layer.colormap = linear.Blues_03
    else:
        layer.colormap = linear.Blues_04
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

interact(f, x=widgets.IntSlider(min=0, max=len(sim_results[0])-2, step=1, value=0))

m.layout.height = '1000px'
m

interactive(children=(IntSlider(value=0, description='x', max=11), Output()), _dom_classes=('widget-interact',…

Map(center=[9.9816, 76.2999], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…